In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *
utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'


2025-07-18 08:34:37.639316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 08:34:37.650234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752849277.659455  121197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752849277.662322  121197 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 08:34:37.675004: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
ncores=1
nmem='10GB'
cluster = PBSCluster(
    cores=ncores, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
    account='P93300041', # Input your project ID here
    walltime='5:00:00', # Amount of wall time
    #interface='ib0', # Interface to use
)

# Scale up
cluster.scale(10)

# Setup your client
client = Client(cluster)

/glade/work/linnia/conda-envs/mlenv/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33617 instead
  warnings.warn(


In [6]:
client.cluster

PBSCluster(5933e382, 'tcp://128.117.208.186:33149', workers=25, threads=25, memory=465.75 GiB)

### setup

In [10]:
def get_biome_param_names(b, u_params, pft_params):
    
    with open("./biome_configs.pkl", "rb") as f:
        biome_configs = pickle.load(f)

    param_names = list(u_params)
    for pft in biome_configs[b]['pfts']:
        pft_param_names = [f"{param}_{pft}" for param in pft_params]
        param_names.extend(pft_param_names)

    return param_names

In [11]:
#======================== set up ============================
# get parameter information
with open(utils_dir+"/param_names.pkl", "rb") as f:
    param_info = pickle.load(f)
u_params = param_info['u_params']
pft_params = param_info['pft_params']

# get biome information
with open("./biome_configs.pkl", "rb") as f:
    biome_configs = pickle.load(f)

# get default parameter set and reset some settings of default parameters
default_params = pd.read_csv('default_params_norm.csv', index_col=False)
default_params.loc[0, ['jmaxb1']] = [0.4]
default_params.loc[0, ['theta_cj']] = [0.7]
default_params.loc[0, ['upplim_destruct_metamorph']] = [1]
default_params.loc[0, ['xl_12']] = [0]

# Build universal_set tensor
universal_set = tf.constant(default_params[u_params].iloc[[0]].to_numpy(),dtype=tf.float64)

x_default = tf.constant(default_params.drop(columns=u_params).iloc[[0]].to_numpy(), dtype=tf.float64)


In [12]:
# get observations 
obs_biome = xr.open_dataset('calibration_obsStatistics_sudokuBiomes_v3.nc')

In [18]:
paths = {
    'lai': '/glade/u/home/linnia/ctsm6_ppe/calibration/calibration_july/emulators_biomelai_compiled/',
    'gpp': '/glade/u/home/linnia/ctsm6_ppe/calibration/calibration_july/emulators_biomegpp_compiled/',
    'biomass': '/glade/u/home/linnia/ctsm6_ppe/calibration/calibration_july/emulators_biomebiomass_compiled/',
    #'lh': '/glade/u/home/linnia/ctsm6_ppe/calibration/emulators_biomeet_compiled/',
    #'sh': '/glade/u/home/linnia/ctsm6_ppe/calibration/emulators_biomesh_compiled/'
}

biomes = [1,2,3,4,5,6,7,8,9,10,11,12] 
param_indices = []
targets = []
stdevs = []
emulators = []  

for b in biomes:
    biome_name = biome_configs[b]['name']
    param_names = get_biome_param_names(b,u_params,pft_params)
    param_ix = [default_params.columns.get_loc(p)for p in param_names]
    
    param_indices.append(param_ix)
    param_indices.append(param_ix)
    param_indices.append(param_ix)
    #param_indices.append(param_ix)
    #param_indices.append(param_ix)
    
    targets.extend([
        tf.convert_to_tensor(1.1*obs_biome.LAI_mean.isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor(1.1*obs_biome.GPP_mean.isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor(1.1*obs_biome.TVC_mean.isel(biome=b).values, dtype=tf.float64),
        #tf.convert_to_tensor(obs_biome.LE_mean.isel(biome=b).values, dtype=tf.float64),
        #tf.convert_to_tensor(obs_biome.SH_mean.isel(biome=b).values, dtype=tf.float64)
    ])
    stdevs.extend([
        tf.convert_to_tensor(obs_biome.LAI_stdev.isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor(obs_biome.GPP_stdev.isel(biome=b).values, dtype=tf.float64),
        tf.convert_to_tensor(obs_biome.TVC_stdev.isel(biome=b).values, dtype=tf.float64),
        #tf.convert_to_tensor(obs_biome.LE_stdev.isel(biome=b).values, dtype=tf.float64),
        #tf.convert_to_tensor(obs_biome.SH_stdev.isel(biome=b).values, dtype=tf.float64),
    ])

    emulators.extend([
        tf.saved_model.load(f"{paths['lai']}{biome_name}"),
        tf.saved_model.load(f"{paths['gpp']}{biome_name}"),
        tf.saved_model.load(f"{paths['biomass']}{biome_name}"),
        #tf.saved_model.load(f"{paths['lh']}{biome_name}"),
        #tf.saved_model.load(f"{paths['sh']}{biome_name}")
    ])
    
targets = tf.stack(targets, axis=0)  
stdevs = tf.stack(stdevs, axis=0)   


In [19]:
# ——— for fast graph execution ———
lengths = [len(arr) for arr in param_indices]     # list of Python ints

emulator_array = []
for m,input_dim in zip(emulators, lengths):
    sig = tf.TensorSpec([None, input_dim], tf.float64)
    f = tf.function(lambda X, _m=m: _m.compiled_predict_f(X)[0], input_signature=[sig])
    emulator_array.append(f)

### Functions

In [21]:

@tf.function
def optimization_step_batch(x, universal_set, emulator_array, param_indices, optimizer, x_default, barrier_strength=1, lambda_penalty=0.01):
    with tf.GradientTape() as tape:
        batch = tf.shape(x)[0]
        u_tiled = tf.tile(universal_set, [batch, 1])   
        x_full = tf.concat([u_tiled, x], axis=1)
        per_model_losses = []
        
        # loop over metrics and biomes in stacked 1D arrays. 
        for i in range(len(emulator_array)):
            model   = emulator_array[i]

            target = tf.reshape(targets[i], (1, -1))
            stdev = tf.reshape(stdevs[i], (1, -1))
            target_tiled = tf.tile(target, [batch, 1])
            stdev_tiled = tf.tile(stdev, [batch, 1])
        
            ix  = param_indices[i]          
            x_biome = tf.gather(x_full, ix, axis=1)
            y_pred = model(x_biome)
            z = tf.abs((y_pred - target_tiled)/stdev_tiled)

            if epsilon is not None: # if biome/variable zscore is < epsilon, it no longer contributes to loss
                mask = tf.cast(z > epsilon, tf.float64)
                loss_b = tf.reduce_sum(mask * tf.square(z), axis=1)
            else:
                loss_b = tf.reduce_sum(tf.square(z), axis=1)
            
            # size [batch]
            per_model_losses.append(loss_b)
            
        loss_per_sample = tf.add_n(per_model_losses)
        data_loss = tf.reduce_mean(loss_per_sample) # overall scalar loss = mean of the batch
        max_z = tf.sqrt(tf.reduce_max(loss_per_sample)) # worst individual biome/init zscore

        # Penalty for moving from default
        # sum over the parameter‐axis (axis=1) → shape [batch]
        x_default_tiled = tf.tile(x_default, [batch, 1])
        sum_abs = tf.reduce_sum(tf.abs(x - x_default_tiled), axis=1)
        penalty = tf.reduce_mean(sum_abs) # mean of batch

        ratio = penalty / lambda_penalty
        factor = tf.maximum(ratio, 1.0) # if the penalty <=lambda, it no longer impacts loss
        total_loss = data_loss * factor
        
        # penalty for being close to/outside of bounds (0,1)
        barrier = tf.reduce_mean(1.0 / (x + 1e-6) + 1.0 / (1.0 - x + 1e-6))
        total_loss = total_loss * (1.0 + barrier_strength * barrier)
        
    grads = tape.gradient(total_loss, [x])
    optimizer.apply_gradients(zip(grads, [x]))
    x.assign(tf.clip_by_value(x, 0.0, 1.0))
    return total_loss, max_z, data_loss, factor, barrier_strength*barrier

In [22]:

def run_optimization(x, universal_set, emulator_array, param_indices, x_default, maxiter, lr, lr_decay_steps, checkpoint_N, checkpoint_dir, epsilon, barrier_strength, lambda_penalty=0.0):
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=lr_decay_steps,
        decay_rate=0.5,
        staircase=True)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    history_total_loss = []
    history_max_z = []
    history_data_loss = []
    history_penalty = []
    history_barrier = []
    
    for step in range(maxiter):

        total_loss, max_z, data_loss, penalty, barrier_loss = optimization_step_batch(x, universal_set, emulator_array, param_indices, optimizer, x_default, barrier_strength, lambda_penalty)

        # log
        history_total_loss.append(total_loss.numpy())
        history_max_z.append(max_z.numpy())
        history_data_loss.append(data_loss.numpy())
        history_penalty.append(penalty.numpy())
        history_barrier.append(barrier_loss.numpy())

        # print updates to screen
        if step % 10 == 0:
            tf.print(f"Step {step:03d}: total={total_loss:.6f} max_z={max_z:.6f}")
    
        # Save a checkpoint every N iterations
        if step % checkpoint_N == 0:
            checkpoint = {
                'step': step,
                'params': x,
                'loss': total_loss,
            }
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_step_{step}.pkl')
            with open(checkpoint_path, 'wb') as f:
                pickle.dump(checkpoint, f)
                
        # early stopping
        if tf.reduce_max(max_z) <= epsilon:
            print(f"Converged at step {step}")
            tf.print(f"Step {step:03d}: total={total_loss:.6f} max_z={max_z:.6f}")
            break
    
    x_opt = x.numpy()
    logs = {
        'total_loss': history_total_loss,
        'max_z': history_max_z,
        'data_loss': history_data_loss,
        'penalty': history_penalty,
        'barrier': history_barrier
    }
    
    return x_opt, logs

### initialization

In [23]:
# initialization: sample from normal priors around default
pft_cols     = [c for c in default_params.columns if c not in u_params]
default_pftparams = default_params[pft_cols].iloc[0].values   

n_inits = 1000
sigma   = 0.05   # spread of your normal draws

np.random.seed(42)   
x0 = np.random.normal(loc=default_pftparams,
                      scale=sigma,
                      size=(n_inits, default_pftparams.size))

# enforce bounds [0,1]
x0 = np.clip(x0, 0.0, 1.0)

x_init = tf.Variable(
    tf.constant(x0, dtype=tf.float64),
    trainable=True,
    name="x"
)

In [11]:
# =========================== save ======================================
n_inits = np.shape(x0)[0]
uset_tiled = np.tile(default_params[u_params].values[0], (n_inits,1))
init_sets = np.concatenate([uset_tiled,x0],axis=1)
init_paramsets = pd.DataFrame(init_sets,columns=default_params.columns.values)

outfile = 'init_sets_defaultPrior.csv'
init_paramsets.to_csv(outfile,index=False)

In [10]:
# or initialize from a previous optimization
with open("./checkpoints_close2default_LHSH/checkpoint_step_500.pkl", "rb") as f:
    c0 = pickle.load(f)

x_init = tf.Variable(
    tf.constant(c0['params'], dtype=tf.float64),
    trainable=True,
    name="x"
)

### calibration

In [24]:
%%time
# Run optimization
epsilon = 0.5
lr = 1e-2
maxiter=2000
lr_decay_steps = 100
checkpoint_N = 50
checkpoint_dir = './checkpoints_sudoku_v3_r1/'
lambda_penalty = 25 # worst sum of delta from default you will tolerate
barrier_strength = 0 # clipping is on

x_opt, logs = run_optimization(x_init, universal_set, emulator_array, param_indices, x_default, 
                               maxiter, lr, lr_decay_steps, checkpoint_N, checkpoint_dir, epsilon, barrier_strength, lambda_penalty)

Step 000: total=727.764153 max_z=30.712824
Step 010: total=231.442910 max_z=17.331942
Step 020: total=140.997806 max_z=13.368867
Step 030: total=88.770623 max_z=10.689221
Step 040: total=63.355608 max_z=9.099816
Step 050: total=47.080095 max_z=7.986366
Step 060: total=38.158215 max_z=7.246628
Step 070: total=32.813163 max_z=6.624344
Step 080: total=29.110294 max_z=6.137682
Step 090: total=26.384810 max_z=5.710884
Step 100: total=24.029293 max_z=5.369998
Step 110: total=22.874828 max_z=5.269688
Step 120: total=21.856989 max_z=5.227124
Step 130: total=20.879369 max_z=5.024934
Step 140: total=20.003088 max_z=4.897691
Step 150: total=19.125280 max_z=4.846968
Step 160: total=18.311749 max_z=4.710680
Step 170: total=17.563025 max_z=4.553131
Step 180: total=16.859997 max_z=4.487599
Step 190: total=16.223212 max_z=4.366897
Step 200: total=15.566992 max_z=4.310786
Step 210: total=15.226489 max_z=4.242230
Step 220: total=14.955865 max_z=4.151288
Step 230: total=14.720096 max_z=4.225676
Step 240:

In [ ]:
# =========================== save ======================================
n_inits = np.shape(x_opt)[0]
uset_tiled = np.tile(default_params[u_params].values[0], (n_inits,1))
opt_sets = np.concatenate([uset_tiled,x_opt],axis=1)
calibrated_paramsets = pd.DataFrame(opt_sets,columns=default_params.columns.values)

outfile = checkpoint_dir+'calibrated_sets_lambda25_071825.csv'
calibrated_paramsets.to_csv(outfile,index=False)

### plot convergence logs

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(logs['total_loss'], label='Total loss')
plt.plot(logs['data_loss'], label='Data loss')
plt.plot(logs['penalty'], linestyle='--',label='Penalty loss')
#plt.plot(logs['barrier'], label='Barrier loss')
plt.xlabel('Step')
plt.ylabel('Loss value')
plt.legend()
plt.grid(True)
plt.title('Loss components over optimization steps')
plt.ylim(0,100)
plt.show()


In [24]:
d_tiled = np.tile(default_params.drop(columns=u_params).values[0], (n_inits,1))
np.sum(np.abs(x_opt - d_tiled),axis=1)

array([35.37247515, 35.80291683, 36.26482413, 34.91029942, 37.31951201,
       35.73246358, 35.32382612, 35.96563349, 37.09204956, 35.63691446,
       35.14781001, 35.57165637, 37.0794535 , 36.67046424, 36.00871308,
       36.30474498, 35.01333341, 37.3339468 , 35.83672696, 35.60952617,
       35.71718743, 35.53491209, 35.7397807 , 37.19296801, 36.25000097,
       35.76375698, 35.36209013, 35.01678331, 35.05688553, 36.22025836,
       36.56990784, 36.85964886, 36.56253986, 35.76800836, 34.60597329,
       35.13937389, 36.42565332, 36.01628154, 34.73287309, 35.24448648,
       35.69435235, 36.08845684, 35.97885046, 35.98806251, 36.37852007,
       35.67985548, 35.77956653, 35.62596604, 35.18934399, 36.86031429,
       35.69546408, 35.94506145, 33.94372023, 35.61717276, 35.74239052,
       34.39420195, 34.57461094, 36.56259174, 35.606587  , 36.73119939,
       36.53415276, 35.16613846, 36.06229644, 36.18256911, 36.30105255,
       36.81562208, 34.57769428, 35.64862675, 36.09250922, 36.11

### Plot posteriors

In [ ]:
full_calibrated_paramsets = pd.read_csv('calibrated_sets_unifRandom.csv')

In [ ]:
# Define LEAP colors
leap_colors = ['#B9D9EB', '#00796B', '#012169']

# Prepare data
pft = 4
pft_param_names = [f"{param}_{pft}" for param in pft_params]
data = full_calibrated_paramsets[pft_param_names]

n = len(pft_param_names)

# Set plot-wide styles
plt.rcParams.update({
    'font.size': 10,         # Font size
    'axes.labelsize': 12,    # Axis label size
    'xtick.labelsize': 8, 
    'ytick.labelsize': 8,
    'axes.linewidth': 0.8,   # Thin frame lines
    'savefig.dpi': 300,
})

fig, axes = plt.subplots(n, n, figsize=(2*n, 2*n))  # Larger, scalable size

for i in range(n):
    for j in range(n):
        ax = axes[i, j]
        if i == j:
            # 1D histogram on the diagonal
            ax.hist(data[pft_param_names[i]], bins=50, color=leap_colors[2], alpha=0.7)
            ax.axvline(default_params[pft_param_names[i]].values)
            ax.set_xlim([0,1])
        else:
            # 2D hexbin on the off-diagonals
            hb = ax.hexbin(
                data[pft_param_names[j]],
                data[pft_param_names[i]],
                gridsize=50,
                #cmap='Blues',
                vmin=0,
                vmax=20,
                mincnt=1,           # Only show bins with data
                linewidths=0
            )
            # Fix hexbin color to LEAP navy color
            #hb.set_cmap('Blues')
            #hb.set_array(hb.get_array()*0+1)  # Uniform color intensity
            #hb.set_edgecolor('none')
            #hb.set_facecolor(leap_colors[2])  # Dark navy
            ax.axvline(default_params[pft_param_names[j]].values)
            ax.axhline(default_params[pft_param_names[i]].values)
            #ax.set_ylim([0,1])
            #ax.set_xlim([0,1])

        # Clean ticks
        if i < n-1:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel(pft_param_names[j], fontsize=10)
        if j > 0:
            ax.set_yticklabels([])
        else:
            ax.set_ylabel(pft_param_names[i], fontsize=10)

plt.tight_layout(pad=0.5)
#plt.savefig('./figs/hexbin_scattermatrix_pftparams_pft'+str(pft)+'.png', dpi = 300)